# 初识StructuredStreaming
设想我们要设计一个交易数据展示系统，实时呈现比特币最近1s钟的成交均价。

我们可以通过交易数据接口以非常低的延迟获得全球各个比特币交易市场的每一笔比特币的成交价，成交额，交易时间。

由于比特币交易事件一直在发生，所以交易事件触发的交易数据会像流水一样源源不断地通过交易接口传给我们。

如何对这种流式数据进行实时的计算呢？我们需要使用流计算工具，在数据到达的时候就立即对其进行计算。

市面上主流的开源流计算工具主要有 Storm, Flink 和 Spark。

其中Storm的延迟最低，一般为几毫秒到几十毫秒，但数据吞吐量较低，每秒能够处理的事件在几十万左右，建设成本高。

Flink是目前国内互联网厂商主要使用的流计算工具，延迟一般在几十到几百毫秒，数据吞吐量非常高，每秒能处理的事件可以达到几百上千万，建设成本低。

Spark通过Spark Streaming或Spark Structured Streaming支持流计算。但Spark的流计算是将流数据按照时间分割成一个一个的小批次(mini-batch)进行处理的，其延迟一般在1秒左右。吞吐量和Flink相当。值得注意的是Spark Structured Streaming 现在也支持了Continous Streaming 模式，即在数据到达时就进行计算，不过目前还处于测试阶段，不是特别成熟。

虽然从目前来看，在流计算方面，Flink比Spark更具性能优势，是当之无愧的王者。但由于Spark拥有比Flink更加活跃的社区，其流计算功能也在不断地完善和发展，未来在流计算领域或许足以挑战Flink的王者地位。

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F 
import time,os,random

#本文主要用小数据测试，设置较小的分区数可以获得更高性能
spark = SparkSession.builder \
        .appName("structured streaming") \
        .config("spark.sql.shuffle.partitions","8") \
        .config("spark.default.parallelism","8") \
        .config("master","local[4]") \
        .enableHiveSupport() \
        .getOrCreate()

sc = spark.sparkContext

23/04/23 15:51:56 WARN Utils: Your hostname, ubuntu-linux-22-04-desktop resolves to a loopback address: 127.0.1.1; using 10.211.55.3 instead (on interface enp0s5)
23/04/23 15:51:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/23 15:51:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 一，Structured Streaming 基本概念
**流计算(Streaming)和批计算(Batch):**

批计算或批处理是处理离线数据。单个处理数据量大，处理速度比较慢。

流计算是处理在线实时产生的数据。单次处理的数据量小，但处理速度更快。


**Spark Streaming 和 Spark Structured Streaming:**

Spark在2.0之前，主要使用的Spark Streaming来支持流计算，其数据结构模型为DStream，其实就是一个个小批次数据构成的RDD队列。

目前，Spark主要推荐的流计算模块是Structured Streaming，其数据结构模型是Unbounded DataFrame，即没有边界的数据表。

相比于 Spark Streaming 建立在 RDD数据结构上面，Structured Streaming 是建立在 SparkSQL基础上，DataFrame的绝大部分API也能够用在流计算上，实现了流计算和批处理的一体化，并且由于SparkSQL的优化，具有更好的性能，容错性也更好。


**source 和 sink:**

source即流数据从何而来。在Spark Structured Streaming 中，主要可以从以下方式接入流数据。

1, Kafka Source。当消息生产者发送的消息到达某个topic的消息队列时，将触发计算。这是structured Streaming 最常用的流数据来源。

2, File Source。当路径下有文件被更新时，将触发计算。这种方式通常要求文件到达路径是原子性(瞬间到达，不是慢慢写入)的，以确保读取到数据的完整性。在大部分文件系统中，可以通过move操作实现这个特性。

3, Socket Source。需要制定host地址和port端口号。这种方式一般只用来测试代码。linux环境下可以用nc命令来开启网络通信端口发送消息测试。

sink即流数据被处理后从何而去。在Spark Structured Streaming 中，主要可以用以下方式输出流数据计算结果。

1, Kafka Sink。将处理后的流数据输出到kafka某个或某些topic中。

2, File Sink。 将处理后的流数据写入到文件系统中。

3, ForeachBatch Sink。 对于每一个micro-batch的流数据处理后的结果，用户可以编写函数实现自定义处理逻辑。例如写入到多个文件中，或者写入到文件并打印。

4， Foreach Sink。一般在Continuous触发模式下使用，用户编写函数实现每一行的处理处理。

5，Console Sink。打印到Driver端控制台，如果日志量大，谨慎使用。一般供调试使用。

6，Memory Sink。输出到内存中，供调试使用。

append mode, complete mode 和 update mode:

这些是流数据输出到sink中的方式，叫做 output mode。

append mode 是默认方式，将新流过来的数据的计算结果添加到sink中。

complete mode 一般适用于有aggregation查询的情况。流计算启动开始到目前为止接收到的全部数据的计算结果添加到sink中。

update mode 只有本次结果中和之前结果不一样的记录才会添加到sink中。


**operation 和 query:**

在SparkSQL批处理中，算子被分为Transformation算子和Action算子。Spark Structured Streaming 有所不同，所有针对流数据的算子都是懒惰执行的，叫做operation。

DataFrame的Action算子(例如show,count,reduce)都不可以在Spark Structured Streaming中使用，而大部分Transformation算子都可以在Structured Streaming中使用(例如select,where,groupBy,agg)。

但也有些操作不可以(例如sort, distinct,某些类型的join操作，以及连续的agg操作等)。

如果要触发执行，需要通过writeStream启动一个query，指定sink，output mode，以及触发器trigger类型。

从一定意义上，可以将writeStream理解成Structured Streaming 唯一的 Action 算子。

Spark Structured Streaming支持的触发器trigger类型主要有以下一些。

1，unspecified。不指定trigger类型，以micro-batch方式触发，当上一个micro-batch执行完成后，将中间收到的数据作为下一个micro-batch的数据。

2，fixed interval micro-batches。指定时间间隔的micro-batch。如果上一个micro-batch在间隔时间内完成，需要等待指定间隔时间。如果上一个micro-batch在间隔时间后才完成，那么会在上一个micro-batch执行完成后立即执行。

3，one-time micro-batch。只触发一次,以micro-batch方式触发。一种在流计算模式下执行批处理的方法。

4，continuous with fixed checkpoint interval。每个事件触发一次，真正的流计算，这种模式目前还处于实验阶段。


event time， processing time 和 watermarking:

event time 是流数据的发生时间，一般嵌入到流数据中作为一个字段。

processing time 是指数据被处理的时间。

Spark Structured Streaming 一般 使用 event time作为 Windows切分的依据，例如每秒钟的成交均价，是取event time中每秒钟的数据进行处理。

考虑到数据存在延迟，如果一个数据到达时，其对应的时间批次已经被计算过了，那么会重新计算这个时间批次的数据并更新之前的计算结果。但是如果这个数据延迟太久，那么可以设置watermarking(水位线)来允许丢弃 processing time和event time相差太久的数据，即延迟过久的数据。注意这种丢弃是或许会发生的，不是一定会丢弃。


at-most once，at-least once 和 exactly once:

这是分布式流计算系统在某些机器发生发生故障时，对结果一致性(无论机器是否发生故障，结果都一样)的保证水平。反应了分布式流计算系统的容错能力。

at-most once，最多一次。每个数据或事件最多被程序中的所有算子处理一次。这本质上是一种尽力而为的方法，只要机器发生故障，就会丢弃一些数据。这是比较低水平的一致性保证。

at-least once，至少一次。每个数据或事件至少被程序中的所有算子处理一次。这意味着当机器发生故障时，数据会从某个位置开始重传。但有些数据可能在发生故障前被所有算子处理了一次，在发生故障后重传时又被所有算子处理了一次，甚至重传时又有机器发生了故障，然后再次重传，然后又被所有算子处理了一次。因此是至少被处理一次。这是一种中间水平的一致性保证。

exactly once，恰好一次。从计算结果看，每个数据或事件都恰好被程序中的所有算子处理一次。这是一种最高水平的一致性保证。

spark structured streaming 在micro-batch触发器类型下，sink是File情况下，可以保证为exactly once的一致性水平。

但是在continuou触发器类型下，只能保证是at-least once的一致性水平。





## 二，word count 基本范例

In [2]:
%%writefile make_streamming_data.py
import random 
import os 
import time 
import shutil
sentences = ["eat tensorflow2 in 30 days","eat pytorch in 20 days","eat pyspark in 10 days"]
data_path = "./data/streamming_data"

if os.path.exists(data_path):
    shutil.rmtree(data_path)
    
os.makedirs(data_path)

for i in range(20):
    line = random.choice(sentences)
    tmp_file = str(i)+".txt"
    with open(tmp_file,"w") as f:
        f.write(line)
        f.flush()
    shutil.move(tmp_file,os.path.join(data_path,tmp_file))
    time.sleep(1)
    

Writing make_streamming_data.py


In [3]:
# 在后台异步生成文件流
import subprocess
cmd = ["python", "make_streamming_data.py"]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

#process.wait() #等待结束


In [4]:
#通过 readStream 创建streaming dataframe
schema = T.StructType().add("value", "string")
data_path = "./data/streamming_data"

dflines = spark \
    .readStream \
    .option("sep", ".") \
    .schema(schema) \
    .csv(data_path)

dflines.printSchema() 
print(dflines.isStreaming) 

root
 |-- value: string (nullable = true)

True


In [5]:
#实施operator转换
dfwords = dflines.select(F.explode(F.split(dflines.value, " ")).alias("word"))
dfwordCounts = dfwords.groupBy("word").count()

In [8]:
#执行query, 注意是异步方式执行, 相当于是开启了后台进程

def foreach_batch_function(df, epoch_id):
    print("Batch: ",epoch_id)
    df.show()

query = dfwordCounts \
    .writeStream \
    .outputMode("complete")\
    .foreachBatch(foreach_batch_function) \
    .start()

# query.awaitTermination() #阻塞当前进程直到query发生异常或者被stop

print(query.isActive)

#60s后主动停止query
time.sleep(30)
query.stop()

print(query.isActive)

23/04/23 16:05:11 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-87858293-7edd-4b04-a199-42c4c6d7e6dc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/23 16:05:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
Batch:  0
+-----------+-----+
|       word|count|
+-----------+-----+
|        eat|   20|
|       days|   20|
|         20|    4|
|tensorflow2|    7|
|         30|    7|
|         10|    9|
|    pyspark|    9|
|         in|   20|
|    pytorch|    4|
+-----------+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/parallels/anaconda3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/parallels/anaconda3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/parallels/anaconda3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 